In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import csv

# Step 1: Load the Dutchie report (assumed to be a CSV file)
# Replace 'dutchie_inventory_report.csv' with the path to your actual report file
try:
    report_df = pd.read_csv('dutchie_inventory_report.csv')
except FileNotFoundError:
    print("Error: The file 'dutchie_inventory_report.csv' was not found. Please provide the correct file path.")
    exit(1)

# Step 2: Filter for edibles
# Assuming the report has a 'Category' column, and edibles are labeled as 'Edibles'
edibles_df = report_df[report_df['Category'].str.contains('Edibles', case=False, na=False)]

# Step 3: Calculate days until expiration
# Assuming the report has an 'Expiration Date' column in the format 'MM/DD/YYYY'
# Convert expiration dates to datetime objects
today = datetime.today()
edibles_df['Expiration Date'] = pd.to_datetime(edibles_df['Expiration Date'], format='%m/%d/%Y')
edibles_df['Days Until Expiration'] = (edibles_df['Expiration Date'] - today).dt.days

# Step 4: Define discount time frames and rules
def apply_discount_rules(days_until_expiration):
    if days_until_expiration <= 0:
        return "Expired - Mark for Destruction", 0
    elif 1 <= days_until_expiration <= 14:
        return "1-14 days until expiration", 50  # 50% discount
    elif 15 <= days_until_expiration <= 30:
        return "15-30 days until expiration", 25  # 25% discount
    elif 31 <= days_until_expiration <= 60:
        return "31-60 days until expiration", 10  # 10% discount
    else:
        return "More than 60 days until expiration", 0  # No discount

# Apply the discount rules to each edible
edibles_df['Status'], edibles_df['Discount (%)'] = zip(*edibles_df['Days Until Expiration'].apply(apply_discount_rules))

# Step 5: Separate expiring items and items to destroy
expiring_items = edibles_df[edibles_df['Days Until Expiration'] > 0]
items_to_destroy = edibles_df[edibles_df['Days Until Expiration'] <= 0]

# Step 6: Display results
print("\n=== Expiring Edibles (Sorted by Days Until Expiration) ===")
if not expiring_items.empty:
    # Sort by days until expiration (ascending)
    expiring_items_sorted = expiring_items.sort_values(by='Days Until Expiration')
    for _, row in expiring_items_sorted.iterrows():
        print(f"Product: {row['Product Name']}, "
              f"Expiration Date: {row['Expiration Date'].strftime('%m/%d/%Y')}, "
              f"Days Until Expiration: {row['Days Until Expiration']}, "
              f"Status: {row['Status']}, "
              f"Discount: {row['Discount (%)']}%")
else:
    print("No edibles are expiring within the discount time frames.")

print("\n=== Edibles to Destroy (Expired) ===")
if not items_to_destroy.empty:
    for _, row in items_to_destroy.iterrows():
        print(f"Product: {row['Product Name']}, "
              f"Expiration Date: {row['Expiration Date'].strftime('%m/%d/%Y')}, "
              f"Days Past Expiration: {abs(row['Days Until Expiration'])}")
else:
    print("No edibles have expired.")

# Step 7: Optionally save the results to CSV files
# Save expiring items
expiring_items.to_csv('expiring_edibles.csv', index=False)
print("\nExpiring edibles saved to 'expiring_edibles.csv'")

# Save items to destroy
items_to_destroy.to_csv('edibles_to_destroy.csv', index=False)
print("Edibles marked for destruction saved to 'edibles_to_destroy.csv'")